# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [17]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [18]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,malvan,16.0667,73.4667,30.41,76,14,5.45,IN,1717123965
1,1,udachny,66.4167,112.4000,9.69,31,3,6.14,RU,1717123965
2,2,adamstown,-25.0660,-130.1015,21.20,73,93,9.75,PN,1717123965
3,3,colonia,50.9333,6.9500,12.94,72,89,1.79,DE,1717123846
4,4,huayin,34.5675,110.0921,30.50,30,91,2.77,CN,1717123965


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [19]:
%%capture --no-display

# Configure the map plot

map_plot = city_data_df.hvplot.points('Lng', 'Lat', geo=True, size="Humidity", scale=3, color="City", alpha=0.5, 
                                      tiles="ESRI", frame_width=1500, frame_height=1000)
# Scale the size of the points for better visualization
# Display the map 
map_plot

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df2 = city_data_df.loc[(city_data_df.Cloudiness == 0) & (city_data_df['Wind Speed'] <= 4.5) & ((city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] <27))]
# Drop any rows with null values
city_data_df2 = city_data_df2.dropna(how='any')

# Display sample data
city_data_df2

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,oltinko'l,40.8011,72.1634,24.91,27,0,2.52,UZ,1717123966
42,42,fort bragg,35.1390,-79.0060,21.57,67,0,0.00,US,1717123969
114,114,yanykurgan,43.9150,67.2481,26.90,17,0,4.30,KZ,1717123976
154,154,illizi,26.4833,8.4667,25.89,14,0,2.97,DZ,1717123980
197,197,buka,40.8108,69.1986,23.94,29,0,2.62,UZ,1717123984
243,243,kerikeri,-35.2268,173.9474,22.17,59,0,3.13,NZ,1717123989
265,265,changji,44.0167,87.3167,23.44,31,0,2.00,CN,1717123991
267,267,zabol,32.2500,67.2500,22.09,18,0,3.45,AF,1717123991
285,285,khadan khak,30.7477,67.7085,23.84,10,0,2.43,PK,1717123993
289,289,damghan,36.1683,54.3480,25.49,27,0,0.30,IR,1717123993


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
columns = ['City', 'Country', 'Lat', 'Lng', 'Humidity']
hotel_df = city_data_df2[columns].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
8,oltinko'l,UZ,40.8011,72.1634,27,
42,fort bragg,US,35.1390,-79.0060,67,
114,yanykurgan,KZ,43.9150,67.2481,17,
154,illizi,DZ,26.4833,8.4667,14,
197,buka,UZ,40.8108,69.1986,29,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'apiKey': geoapify_key,
    'limit': 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    longitude = row.Lng
    Latitude = row.Lat
    Humidity = row.Humidity
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{Latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{Latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
oltinko'l - nearest hotel: No hotel found
fort bragg - nearest hotel: Airborne Inn Lodging
yanykurgan - nearest hotel: No hotel found
illizi - nearest hotel: بوناقة
buka - nearest hotel: No hotel found
kerikeri - nearest hotel: Avalon Resort
changji - nearest hotel: No hotel found
zabol - nearest hotel: No hotel found
khadan khak - nearest hotel: No hotel found
damghan - nearest hotel: هتل جهانگردى دامغان
neftcala - nearest hotel: No hotel found
winnemucca - nearest hotel: The Overland Hotel
marsa alam - nearest hotel: Blue House Hotel
berezovyy - nearest hotel: No hotel found
la cruz - nearest hotel: Hotel Las Palmas
osuna - nearest hotel: Hotel Palacio Marques De La Gomera
tamanrasset - nearest hotel: فندق الأمان
mesquite - nearest hotel: Holiday Inn Express & Suites
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by Wyndham


,City,Country,Lat,Lng,Humidity,Hotel Name
8,oltinko'l,UZ,40.8011,72.1634,27,No hotel found
42,fort bragg,US,35.1390,-79.0060,67,Airborne Inn Lodging
114,yanykurgan,KZ,43.9150,67.2481,17,No hotel found
154,illizi,DZ,26.4833,8.4667,14,بوناقة
197,buka,UZ,40.8108,69.1986,29,No hotel found
243,kerikeri,NZ,-35.2268,173.9474,59,Avalon Resort
265,changji,CN,44.0167,87.3167,31,No hotel found
267,zabol,AF,32.2500,67.2500,18,No hotel found
285,khadan khak,PK,30.7477,67.7085,10,No hotel found
289,damghan,IR,36.1683,54.3480,27,هتل جهانگردى دامغان


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points('Lng', 'Lat', geo=True, size="Humidity", scale=3, color="City", alpha=0.5, 
                                      tiles="OSM", frame_width=1500, frame_height=1000, hover_cols = ['Hotel Name'])

# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)